In [ ]:
# Import some libraries first... 
import os
import re
import mne
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mne.stats import permutation_cluster_test
from mne.stats import permutation_cluster_1samp_test


from time import time
from joblib import Parallel, delayed

In [ ]:
data_dir = '/add/your/path/here/'

output_dir = '/add/your/path/here/'
save_plot_path = '/add/your/path/here/' # testest
save_Morlet_mean_theta_path = '/add/your/path/here/' # testest


# Specify which condition is used 
NOVEL = False
FAMIL = True 

In [ ]:
# Define what epoch you want to analyse and which EEG task to use 
epoch = "DM_rem" #DM_rem or Hit_recog # Hit_trig can be used for landmark 
day = "DAY1"
EEGtask = "memTask" #memTasklandmark

participant_ids = []
eeg_files = {"DAY1": [], "DAY2": []}

pattern = r"sub-(\d+)_"
data_dirs = data_dir + day

In [ ]:
# Set the parameters for the Morlet wavelet
freqs = np.arange(3, 30, 1)  
n_cycles = 5  
tf_power = []

In [ ]:
for participant_folder in os.listdir(data_dirs):
    
    participant_id = participant_folder.split('-')[1]
    participant_ids.append(participant_id)
    eeg_file = os.path.join(data_dirs, participant_folder)
    
    eeg_files[day].append(eeg_file)
print(eeg_files[day])

In [ ]:
def run_Morlet(eeg_preproc_file):
    
    epochs = mne.read_epochs(eeg_preproc_file)
    power, itc = mne.time_frequency.tfr_morlet(epochs[epoch], n_cycles=n_cycles, return_itc=True,
                                  freqs=freqs, decim=3, average=True)
    power.apply_baseline(mode="zscore", baseline=(-0.7, -0.2))
    itc.apply_baseline(mode="zscore", baseline=(-0.7, -0.2))
    
    return (power, itc) 

In [ ]:
# Run the Morlet Wavelet function in parallel port 
t0 = time()
power_itc_pairs = Parallel(n_jobs=4)(delayed(run_Morlet)(eeg_preproc_file) for eeg_preproc_file in eeg_files[day])
tf_power, tf_itc = zip(*power_itc_pairs)  
t1 = time()
print(f'This MNE Morlet Wavelet code run in parallel took {t1 - t0} seconds to run')


for i, pair in enumerate(power_itc_pairs):
    participant_id = os.path.basename(eeg_files[day][i]).split("_")[0]  
    participant_output_dir = os.path.join(output_dir, participant_id, day)
    os.makedirs(participant_output_dir, exist_ok=True)  
    
    power, itc = pair
    
    power_file = os.path.join(participant_output_dir, f"{participant_id}_{epoch}power.npy")
    itc_file = os.path.join(participant_output_dir, f"{participant_id}_{epoch}itc.npy")
    
    np.save(power_file, power)
    np.save(itc_file, itc)